# Scala mcsapi connector Test
## Testing the scala mcsapi connector
Include the necessary libraries for testing:

In [ ]:
import com.mariadb.columnstore.api.connector.ColumnStoreExporter
import java.math.{BigDecimal,BigInteger,MathContext}
import org.apache.spark.sql.SQLContext
import org.junit.Test;
import java.util.Properties
import org.junit.Assert._
import java.sql.{DriverManager,Connection,Date,Timestamp,PreparedStatement,ResultSet,SQLException}
val sqlContext = new SQLContext(sc)
import sqlContext.implicits._

Create the test table, through JDBC.

In [ ]:
val url = "jdbc:mysql://columnstore:3306/test"
var connectionProperties = new Properties()
connectionProperties.put("user", "root")
connectionProperties.put("driver", "org.mariadb.jdbc.Driver")
var connection:Connection = _
try {
    connection = DriverManager.getConnection(url, connectionProperties)
    val statement = connection.createStatement
    val rs1 = statement.executeQuery("""DROP TABLE IF EXISTS scalatest""")
    val rs2 = statement.executeQuery("""
    CREATE TABLE scalatest (
    uint64 bigint unsigned,
    int64 bigint,
    uint32 int unsigned,
    int32 int,
    uint16 smallint unsigned,
    int16 smallint,
    uint8 tinyint unsigned,
    `int8` tinyint,
    f float,
    d double,
    ch4 char(5),
    vch30 varchar(30),
    dt date,
    dtm datetime,
    dc decimal(18),
    tx text,
    bit tinyint(1),
    mathInt bigint unsigned,
    dc2 decimal(18,9))
    ENGINE=columnstore""")
} catch {
    case e: Exception => e.printStackTrace
}

Create the test DataFrame to insert into scalatest.

In [ ]:
val testDF = Seq(
  (1L, 2L, 3L, 4, 5, 6, 7, 8, 1.234F, 2.34567F, "ABCD", "Hello World", Date.valueOf("2017-09-08"), Timestamp.valueOf("2017-09-08 13:58:23"), new BigDecimal(123), "Hello World Longer", true, new BigInteger("9223372036854775807"), new BigDecimal("-0.000000001",MathContext.UNLIMITED)),
  (0L, -9223372036854775806L, 0L, -2147483646, 0, -32766, 0, -126, 1.234F, 2.34567F, "A", "B", Date.valueOf("1000-01-01"), Timestamp.valueOf("1000-01-01 00:00:00"), new BigDecimal(-123), "C", false, new BigInteger("18446744073709551613"), new BigDecimal("100000000.999999999",MathContext.UNLIMITED)),
  (9223372036854775807L, 9223372036854775807L, 4294967293L, 2147483647, 65533, 32767, 253, 127, 1.234F, 2.34567F, "ZYXW", "012345678901234567890123456789", Date.valueOf("9999-12-31"), Timestamp.valueOf("9999-12-31 23:59:59"), new BigDecimal(123), "012345678901234567890123456789", true, new BigInteger("2342"), new BigDecimal("23.42"))
).toDF("uint64", "int64", "uint32", "int32", "uint16", "int16", "uint8", "int8", "f", "d", "ch4", "vch30", "dt", "dtm", "dc", "tx", "bit", "mathInt", "dc2")

testDF.printSchema()
testDF.registerTempTable("df")
sqlContext.sql("SELECT * from df").show()

 and is inserted into MariaDB CoumnStore through ColumnstoreExporter in database "test" and created table "scalatest".

In [ ]:
ColumnStoreExporter.export("test","scalatest",testDF)

Validate the correctness of the insert

In [ ]:
def verifyAllTypes(conn: Connection, id: Long, expected: String) : Unit = {
        val QUERY_ALL_TYPES = "select uint64, int64, uint32, int32, uint16, int16, uint8, `int8`, " +
                "f, d, ch4, vch30, dt, dtm, dc, tx, bit, mathInt, dc2 from scalatest where uint64 = ?"
        var stmt: PreparedStatement = null
        var rs: ResultSet = null
        try {
           stmt = conn.prepareStatement(QUERY_ALL_TYPES)
           stmt.setLong(1, id)
           rs = stmt.executeQuery()
           assertTrue(rs.next())
           var str = new StringBuffer()
           val colCount = stmt.getMetaData().getColumnCount()
           for (i <- 1 to colCount){
               if (i>1) str.append(", ")
               str.append(rs.getObject(i))
           }
           assertEquals(expected, str.toString())
        }
        catch {
            case e: SQLException => println("Error while validating all_types results for id: " + id + ", error:" + e)
        }
        finally {
            rs.close
            stmt.close
        }
    }

verifyAllTypes(connection, 1L, "1, 2, 3, 4, 5, 6, 7, 8, 1.234, 2.345669984817505, ABCD, Hello World, 2017-09-08, 2017-09-08 13:58:23.0, 123, Hello World Longer, true, 9223372036854775807, -1E-9");
verifyAllTypes(connection, 0L, "0, -9223372036854775806, 0, -2147483646, 0, -32766, 0, -126, 1.234, 2.345669984817505, A, B, 1000-01-01, 1000-01-01 00:00:00.0, -123, C, false, 18446744073709551613, 100000000.999999999");
verifyAllTypes(connection, 9223372036854775807L, "9223372036854775807, 9223372036854775807, 4294967293, 2147483647, 65533, 32767, 253, 127, 1.234, 2.345669984817505, ZYXW, 012345678901234567890123456789, 9999-12-31, 9999-12-31 23:59:59.0, 123, 012345678901234567890123456789, true, 2342, 23.420000000");

In [ ]:
connection.close